In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import string
import chromedriver_binary

pd.options.display.float_format = '{_.0f}'.format

In [2]:
is_link = 'https://finance.yahoo.com/quote/AAPL/financials?p=AAPL'
driver = webdriver.Chrome('C:/Users/Andrew S/Documents/Trabajo/Software/chromedriver')
driver.get(is_link)
html = driver.execute_script('return document.body.innerHTML;')
soup = BeautifulSoup(html,'lxml')

<ipython-input-2-f8bbd73774a0>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Andrew S/Documents/Trabajo/Software/chromedriver')


The above code will open a Chrome dummy page in order to pull all the data whithin the is:link website body. Yahoo Finance operates on JavaScript, so we ned to run the code through this metod to pull all the data and save it as an static website. 

In [3]:
close_price = [entry.text for entry in soup.find_all('span',{'class':'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})]

In [4]:
close_price

['156.81']

Using the Inspect Tool of Google Chrom we hover ober the price and get the info about the location of the stock price. The above code, searches for the 'span' tag within all of the HTML code and loocks for the class attribute that matches the one entered. 

#### Pulling financial statement data

We find out that all the financial statements are located on the div containers with a common class attribute of 'D(tbr)'. In the next example there will ve additional pieces of data in the class attirbute, but as long as the first portion matches what we're searching it will pull the data.

In [5]:
features = soup.find_all('div', class_='D(tbr)')
features

[<div class="D(tbr) C($primaryColor)" data-reactid="37"><div class="D(ib) Fw(b) Ta(start) Px(15px)--mv2 Px(10px) W(247px)--mv2 W(222px) Bxz(bb) Bdendw(1px) Bdstartw(1px) Bdbw(1px) Bdends(s) Bdstarts(s) Bdbs(s) Bdc($seperatorColor) Py(6px) Pos(st) Start(0) Bgc($lv2BgColor)" data-reactid="38"><span data-reactid="39">Breakdown</span><div class="W(3px) Pos(a) Start(100%) T(0) H(100%) Bg($pfColumnFakeShadowGradient) Pe(n) Pend(5px)" data-reactid="40"></div></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(ib) Fw(b) Tt(u) Bgc($lv1BgColor)" data-reactid="41"><span data-reactid="42">ttm</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(ib) Fw(b)" data-reactid="43"><span data-reactid="44">9/30/2021</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(ib) Fw(b) Bgc($lv1BgColor)" data-reactid="45"><span data-reactid="46">9/30/2020</span></div><div clas

The next code is able to pull only the finantial data in features

In [6]:
headers = []
label_list = []
final = []
index = 0
#create headers
for item in features[0].find_all('div', class_ = 'D(ib)'):
    headers.append(item.text)
headers

['Breakdown', 'ttm', '9/30/2021', '9/30/2020', '9/30/2019', '9/30/2018']

In [7]:
#statement contents
temp_list = []
while index<= len(features) -1:
    #filter for each line of the statement
    temp = features[index].find_all('div', class_='D(tbc)')
    for line in temp:
        #each item adding to a temporary list
        temp_list.append(line.text)
    #temp_list added to final list
    final.append(temp_list)
    #clear temp_list
    temp_list = []
    index+=1
df = pd.DataFrame(final[1:])
df.columns = headers

In [8]:
df

,Breakdown,ttm,9/30/2021,9/30/2020,9/30/2019,9/30/2018
0,Total Revenue,"365,817,000","365,817,000","274,515,000","260,174,000","265,595,000"
1,Cost of Revenue,"212,981,000","212,981,000","169,559,000","161,782,000","163,756,000"
2,Gross Profit,"152,836,000","152,836,000","104,956,000","98,392,000","101,839,000"
3,Operating Expense,"43,887,000","43,887,000","38,668,000","34,462,000","30,941,000"
4,Operating Income,"108,949,000","108,949,000","66,288,000","63,930,000","70,898,000"
5,Net Non Operating Interest Income Expense,"198,000","198,000","890,000","1,385,000","2,446,000"
6,Other Income Expense,"60,000","60,000","-87,000","422,000","-441,000"
7,Pretax Income,"109,207,000","109,207,000","67,091,000","65,737,000","72,903,000"
8,Tax Provision,"14,527,000","14,527,000","9,680,000","10,481,000","13,372,000"
9,Net Income Common Stockholders,"94,680,000","94,680,000","57,411,000","55,256,000","59,531,000"


Now we change all the numbers on the page as strings

In [9]:
#function to make all values numerical
def convert_to_numeric(column):
    first_col = [i.replace(',','') for i in column]
    second_col = [i.replace('-','') for i in first_col]
    final_col = pd.to_numeric(second_col)
    
    return final_col

In [10]:
for column in headers[1:]:
    
    df['column'] = convert_to_numeric(df[column])
final_df = df.fillna('-')

In [11]:
df['ttm']

0     365,817,000
1     212,981,000
2     152,836,000
3      43,887,000
4     108,949,000
5         198,000
6          60,000
7     109,207,000
8      14,527,000
9      94,680,000
10     94,680,000
11              -
12              -
13              -
14              -
15    108,949,000
16    256,868,000
17     94,680,000
18     94,680,000
19      2,843,000
20      2,645,000
21        198,000
22    111,852,000
23    123,136,000
24    212,981,000
25     11,284,000
26     94,680,000
27    123,136,000
28              0
29              0
Name: ttm, dtype: object

# Scrapping BTC

In [12]:
is_link = 'https://finance.yahoo.com/quote/BTC-USD/history?period1=1480291200&period2=1638057600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true'
driver = webdriver.Chrome('C:/Users/Andrew S/Documents/Trabajo/Software/chromedriver')
driver.get(is_link)
html = driver.execute_script('return document.body.innerHTML;')
soup = BeautifulSoup(html,'lxml')

<ipython-input-12-13be029421b1>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Andrew S/Documents/Trabajo/Software/chromedriver')


In [13]:
close_price = [entry.text for entry in soup.find_all('span',{'class':'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})]

In [14]:
close_price

['57,488.80']

In [15]:
features = soup.find_all('td', class_='Py(10px) Ta(start) Pend(10px)')
features

[<td class="Py(10px) Ta(start) Pend(10px)" data-reactid="52"><span data-reactid="53">Nov 28, 2021</span></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="61"><span data-reactid="62">Nov 27, 2021</span></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="76"><span data-reactid="77">Nov 26, 2021</span></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="91"><span data-reactid="92">Nov 25, 2021</span></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="106"><span data-reactid="107">Nov 24, 2021</span></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="121"><span data-reactid="122">Nov 23, 2021</span></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="136"><span data-reactid="137">Nov 22, 2021</span></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="151"><span data-reactid="152">Nov 21, 2021</span></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="166"><span data-reactid="167">Nov 20, 2021</spa

# Second Method

In [1]:
import pandas as pd
import time
import datetime

In [20]:
pd.set_option('display.notebook_repr_html', True)
ticker = 'AAPL'
period1 = int(time.mktime(datetime.datetime(2021,10,28,20,1).timetuple()))
period2 = int(time.mktime(datetime.datetime(2021,11,28,20,1).timetuple()))
interval = '1d' #1w 1m 1y
url_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
# url_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&elements=history&includeAdjustedClose=true'
print(url_string)
data = pd.read_csv(url_string,sep=r',')
data

https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=1635469260&period2=1638151260&interval=1d&events=history&includeAdjustedClose=true


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-10-29,147.220001,149.940002,146.410004,149.800003,149.581696,124850400
1,2021-11-01,148.990005,149.699997,147.800003,148.960007,148.742920,74588300
2,2021-11-02,148.660004,151.570007,148.649994,150.020004,149.801376,69122000
3,2021-11-03,150.389999,151.970001,149.820007,151.490005,151.269241,54511500
4,2021-11-04,151.580002,152.429993,150.639999,150.960007,150.740005,60394600
5,2021-11-05,151.889999,152.199997,150.059998,151.279999,151.279999,65414600
6,2021-11-08,151.410004,151.570007,150.160004,150.440002,150.440002,55020900
7,2021-11-09,150.199997,151.429993,150.059998,150.809998,150.809998,56787900
8,2021-11-10,150.020004,150.130005,147.850006,147.919998,147.919998,65187100
9,2021-11-11,148.960007,149.429993,147.679993,147.869995,147.869995,41000000


In [18]:
data['Indicador1'] = data['High']*data['Low']

In [19]:
data

,Date,Open,High,Low,Close,Adj Close,Volume,Indicador1
0,2021-10-29,147.220001,149.940002,146.410004,149.800003,149.581696,124850400,21952.716293
1,2021-11-01,148.990005,149.699997,147.800003,148.960007,148.742920,74588300,22125.660006
2,2021-11-02,148.660004,151.570007,148.649994,150.020004,149.801376,69122000,22530.880631
3,2021-11-03,150.389999,151.970001,149.820007,151.490005,151.269241,54511500,22768.146614
4,2021-11-04,151.580002,152.429993,150.639999,150.960007,150.740005,60394600,22962.053993
5,2021-11-05,151.889999,152.199997,150.059998,151.279999,151.279999,65414600,22839.131245
6,2021-11-08,151.410004,151.570007,150.160004,150.440002,150.440002,55020900,22759.752857
7,2021-11-09,150.199997,151.429993,150.059998,150.809998,150.809998,56787900,22723.584447
8,2021-11-10,150.020004,150.130005,147.850006,147.919998,147.919998,65187100,22196.722140
9,2021-11-11,148.960007,149.429993,147.679993,147.869995,147.869995,41000000,22067.820320


In [6]:
print(data.columns.tolist())

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


In [11]:
import numpy as np
Low = np.array(data['Low'].tolist())

In [12]:
Low

array([146.410004, 147.800003, 148.649994, 149.820007, 150.639999,
       150.059998, 150.160004, 150.059998, 147.850006, 147.679993,
       147.479996, 149.429993, 149.339996, 150.990005, 153.050003,
       156.529999, 161.      , 159.059998, 159.639999, 156.360001])

In [13]:
Low*2

array([292.820008, 295.600006, 297.299988, 299.640014, 301.279998,
       300.119996, 300.320008, 300.119996, 295.700012, 295.359986,
       294.959992, 298.859986, 298.679992, 301.98001 , 306.100006,
       313.059998, 322.      , 318.119996, 319.279998, 312.720002])

In [14]:
data['Low']*2

0     292.820008
1     295.600006
2     297.299988
3     299.640014
4     301.279998
5     300.119996
6     300.320008
7     300.119996
8     295.700012
9     295.359986
10    294.959992
11    298.859986
12    298.679992
13    301.980010
14    306.100006
15    313.059998
16    322.000000
17    318.119996
18    319.279998
19    312.720002
Name: Low, dtype: float64

In [15]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-10-29,147.220001,149.940002,146.410004,149.800003,149.581696,124850400
1,2021-11-01,148.990005,149.699997,147.800003,148.960007,148.742920,74588300
2,2021-11-02,148.660004,151.570007,148.649994,150.020004,149.801376,69122000
3,2021-11-03,150.389999,151.970001,149.820007,151.490005,151.269241,54511500
4,2021-11-04,151.580002,152.429993,150.639999,150.960007,150.740005,60394600
5,2021-11-05,151.889999,152.199997,150.059998,151.279999,151.279999,65414600
6,2021-11-08,151.410004,151.570007,150.160004,150.440002,150.440002,55020900
7,2021-11-09,150.199997,151.429993,150.059998,150.809998,150.809998,56787900
8,2021-11-10,150.020004,150.130005,147.850006,147.919998,147.919998,65187100
9,2021-11-11,148.960007,149.429993,147.679993,147.869995,147.869995,41000000


In [53]:
import numpy as np
Volume = np.array(data['Volume'].tolist()) *12


In [36]:
print(date2['Low'].tolist())

[146.410004, 147.800003, 148.649994, 149.820007, 150.639999, 150.059998, 150.160004, 150.059998, 147.850006, 147.679993, 147.479996, 149.429993, 149.339996, 150.990005, 153.050003, 156.529999, 161.0, 159.059998, 159.639999, 156.360001]


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       20 non-null     object 
 1   Open       20 non-null     float64
 2   High       20 non-null     float64
 3   Low        20 non-null     float64
 4   Close      20 non-null     float64
 5   Adj Close  20 non-null     float64
 6   Volume     20 non-null     int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.2+ KB


In [44]:
data.items()

<generator object DataFrame.items at 0x00000128DA7EB9E0>